In [ ]:
# ############################################################################ #
# THE FOLLOWING CELL IS REQUIRED ONLY TO TEST LOCAL UNBUILT VERSIONS OF ISLATU
# ############################################################################ #
# Necessary to reload islatu when making changes to a local version.
import importlib
from os import chdir

chdir("/Users/richardbrearton/Documents/Code/islatu")


In [ ]:
# Set up the notebook
%load_ext autoreload
%autoreload 2

# Uncomment to see what <autoreload 2> is doing.
# %autoreload?

In [ ]:
# First, we need some preliminary setup.
# We will need a list of the paths to the data files.
data_dir = "/Users/richardbrearton/Documents/Data/i07_data/DCD/tom_arnold_data/"
run_numbers = list(range(404875, 404883))
data_files = [data_dir + "i07-" + str(i) + ".nxs" for i in run_numbers]

# Was this data recorded with the diffractometer in the DCD configuration?
is_DCD_data = True

if is_DCD_data:
    DCD_normalization_file = data_dir + "404863.dat"


In [ ]:
# Now we need to load the data into an Isaltu Profile object.
from islatu.profile import Profile

# We're loading .nxs files acquired at beamline I07, so we'll need to make use
# of Islatu's I07 .nxs parser
from islatu.io import i07_nxs_parser
import islatu

importlib.reload(islatu.profile)
importlib.reload(islatu.io)


# Note: at this stage, the various detector images will be loaded into RAM.
# As reflectometry profiles can contain a large number of high resolution
# detector frames, the following line of code may take a long time to execute.
profile = Profile.fromfilenames(data_files, i07_nxs_parser)


In [ ]:
# A reflectivity profile is made up of a series of scans, each of which contains
# a number of images that we have just loaded into RAM. Now we can plot any of
# raw detector frames. Matplotlib is rubbish, so we're going to use plotly for
# data visualization. Below is a convenience function that we'll use to render
# raw images.

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


def plot_array(input_array, imshow_data_map='log', colorscale='Jet'):
    """
    Simple convenience function for plotting detector frames.

    Args:
        input_array (:py:attr:`array_like`):
            An array to be plotted
        imshow_data_map (:py:attr:`str`, optional):
            A string specifying how input_array will be mapped before it is
            displayed. Options are 'log', 'sqrt' and 'linear'.

    """
    # Currently implemented ways to map the input array for the imshow.
    imshow_data_maps = {"linear": lambda img: np.copy(img),
                        "log": lambda img: np.log(np.copy(img)+0.1),
                        "sqrt": lambda img: np.sqrt(np.copy(img))}

    # Map the input_array to create the array that will be rendered.
    arr_to_display = imshow_data_maps[imshow_data_map](input_array)

    fig = make_subplots(rows=3, cols=1, specs=[[{"type": "heatmap"}],
                                               [{"type": "xy"}],
                                               [{"type": "xy"}]],
                        subplot_titles=("Raw detector image",
                                        "Mean value of pixels along x",
                                        "Mean value of pixels along y"))

    fig.append_trace(go.Heatmap(z=arr_to_display, colorscale=colorscale), 
        row=1, col=1)

    # Average pixel value as a function of x, y
    input_array_x_mean = np.mean(input_array, axis=0)
    input_array_y_mean = np.mean(input_array, axis=1)

    fig.append_trace(go.Scatter(y=input_array_x_mean), row=2, col=1)
    fig.append_trace(go.Scatter(y=input_array_y_mean), row=3, col=1)

    # Now set the figure title
    fig.update_layout(
        height=1000, title_text="Raw Detector Data", showlegend=False)

    # Update xaxis titles
    fig.update_xaxes(title_text="Detector x-axis", row=1, col=1)
    fig.update_xaxes(title_text="x-pixel", row=2, col=1)
    fig.update_xaxes(title_text="y-pixel", row=3, col=1)

    # Update yaxis titles
    fig.update_yaxes(title_text="Detector y-axis", row=1, col=1)
    fig.update_yaxes(title_text="Mean intensity", row=2, col=1)
    fig.update_yaxes(title_text="Mean intensity", row=3, col=1)

    fig.show()


In [ ]:
# Lets define a function to make grabbing images from our profile as simple as
# possible. We could do this in one line, but it would get pretty annoying.

def get_image_array(scan_number, img_number):
    scan_object = profile.scans[scan_number]
    img_object = scan_object.images[img_number]

    frame = img_object.n
    return frame
    

In [ ]:
# Now we can very simply inspect any of the raw detector images acquired as a
# part of our profile.

# Enter the scan/image numbers of interest.
scan_number = 0
img_number = 0

# Load the img_array
img_array = get_image_array(scan_number, img_number)

# How do you want your data to be rescaled for visualization purposes?
# Options are: "log", "sqrt" and "None" (None keeps the raw data)
imshow_data_map = "log"

plot_array(img_array, imshow_data_map)


In [ ]:
#TODO: auto cropping should go here. 

# The raw detector frame is typically much larger than the area containing
# scattered intensity. As the following operations can be computationally
# demanding, it's best to start by cropping the image down to something more
# manageable.

from islatu.cropping import crop_2d

# The pixel coordinates of the corners of the cropped image.
x_start = 1150
x_end = 1280
y_start = 150
y_end = 300

profile.crop(crop_2d, {
            'x_start': profile.scans[0].metadata.roi_1_x1,
            'x_end': profile.scans[0].metadata.roi_1_x2,
            'y_start': profile.scans[0].metadata.roi_1_y1,
            'y_end': profile.scans[0].metadata.roi_1_y2
})

# Perform the cropping.
# profile.crop(crop_2d, {'x_start': x_start, 'x_end': x_end,
#                        'y_start': y_start, 'y_end': y_end})


In [ ]:
# Now plot the cropped image; make sure the output looks reasonable. It is not
# necessary for the peak to be perfectly centered, but it should be fairly 
# close.
scan_number = 7
img_number = 23
imshow_data_map = "linear"
cropped_img_array = get_image_array(scan_number, img_number)
cropped_img_array = profile.scans[scan_number].images[img_number].n

plot_array(cropped_img_array, imshow_data_map)


In [ ]:
# Now we want to subtract the background. In an experiment, a user will specify
# two regions of interest in GDA. ROI_1 contains the reflected intensity, while
# ROI_2 indicates a background region. 

# Here, will will specify our own ROI_2 so we can play around with it.
roi_2 = {'x_start': 1100, 'x_end': 1180,
         'y_start': 150, 'y_end': 300}


# Or, alternatively, we can access the roi_2 that was set in the experiment.
roi_2 = {
    'x_start': profile.scans[0].metadata.roi_2_x1,
    'x_end': profile.scans[0].metadata.roi_2_x2,
    'y_start': profile.scans[0].metadata.roi_2_y1,
    'y_end': profile.scans[0].metadata.roi_2_y2
}

# Carry out the background subtraction.
mean_bkg_per_pixel = profile.bkg_sub(None, roi_2)
print(mean_bkg_per_pixel)


In [ ]:
# We can quickly plot our beam profile to make sure that the background looks
# properly subtracted.

# Scan and image numbers of interest.
scan_number = 7
img_number = 23

# Get the image array for the scan of interest.
arr = get_image_array(scan_number, img_number)
arr_along_x = np.mean(arr, axis=0)

arr_e = profile.scans[scan_number].images[img_number].array_e
arr_e_along_x = np.mean(arr_e, axis=0)

# Prepare the figure.
title = "Scan {}, Image {}".format(scan_number, img_number)
fig = go.Figure().update_layout(title=title)
fig.add_trace(go.Scatter(y=arr_along_x))
fig.add_trace(go.Scatter(y=arr_e_along_x))
fig.show()


In [ ]:
# !Experimental code cell!
# Towards the end of a scan, there's a tendancy for signal to become rather lost
# in the noise. We can try to extract more signal by wavelet transforming our
# data, setting the high frequency wavelet coefficients to 0 and transforming
# back.

import pywt

# We're just testing, don't mess with the profile yet.
arr_along_x_copy = arr_along_x.copy()

wavelet_choice = "sym4"


In [ ]:
import matplotlib.pyplot as plt
def wavelet_denoise(raw_data, wavelet_choice, threshold):
    """
    A function to perform some simple wavelet denoising on some raw input data.
    """
    w = pywt.Wavelet(wavelet_choice)
    maxlev = pywt.dwt_max_level(len(raw_data), w.dec_len)
    # maxlev = 2 # Override if desired
    print("maximum level is " + str(maxlev))


    # Decompose into wavelet components, to the level selected:
    coeffs = pywt.wavedec(raw_data, wavelet_choice, level=maxlev)

    # first find the largest coefficient
    imax, jmax = 0, 0
    max_coef = 0
    for i in range(1, len(coeffs)):
        for j in range(len(coeffs[i])):
            if coeffs[i][j] >= max_coef:
                max_coef = coeffs[i][j]

    print(max_coef)

    #cA = pywt.threshold(cA, threshold*max(cA))
    plt.figure()
    for i in range(0, len(coeffs)):

        plt.subplot(maxlev+1, 1, i+1)
        plt.plot(coeffs[i])
        if i < maxlev:
            coeffs[i] = pywt.threshold(coeffs[i], threshold*max_coef)
        else:  
            # # always kill ultra high frequency nonsense
            # coeffs[i] = pywt.threshold(coeffs[i], 1*max_coef)
            coeffs[i] = pywt.threshold(coeffs[i], threshold*max_coef)
        plt.plot(coeffs[i])

    plt.show()
    return coeffs


In [ ]:
discrete_wav_choice = "sym4"
wavelet_coeffs = wavelet_denoise(arr_along_x_copy, discrete_wav_choice, 0.5)


In [ ]:
datarec = pywt.waverec(wavelet_coeffs, discrete_wav_choice)

plt.plot(arr_along_x_copy)
plt.show()
plt.plot(datarec)
plt.show()

plt.plot(arr_along_x_copy)
plt.plot(datarec)
plt.show()


In [ ]:
pywt.wavelist(kind='continuous')


In [ ]:

# try a continuous wavelet transform
import plotly.express as px
wavelet_choice = 'gaus8'
scales = np.arange(1, len(arr_along_x_copy))
#scales = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256, 512])

coef, freqs = pywt.cwt(arr_along_x_copy, scales, wavelet_choice)

fig = go.Figure(data=[go.Surface(z=np.abs(coef))])

fig.update_layout(title='Continuous wavelet ({}) transform'.format(
    wavelet_choice), 
                  autosize=False,
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()


In [ ]:
# This'll come in handy for plotting the various XRR curves we come across.

def plot_xrr_curve(refl_profile, title, is_log=True):
    """
    Convenience function for plotting simple XRR curves.
    """
    fig = go.Figure().update_layout(title=title,
                                    xaxis_title='Q/Å', yaxis_title='R')

    fig.add_trace(go.Scatter(x=(profile.q),
                             y=(profile.R), error_y={
        "type": 'data',
        "array": (profile.R_e),
        "visible": True},
        name="Islatu"))

    if is_log:
        fig.update_yaxes(type="log")

    fig.show()


In [ ]:
# This is kept separate so that one can replot graphs without overwriting the
# title.
title = "Bkg Corrections."

In [ ]:
# Now that the background has been subtracted we can view the current state of
# the reflectivity profile.
plot_xrr_curve(profile, title=title)


In [ ]:
# Clearly, the above plot leaves a lot to be desired. Now it is necessary to 
# carry out a series of corrections on the data. Firstly, if this data was
# acquired using the DCD setup, then we need to specify a DCD normalization
# file that will have been collected before this profile.

from islatu.corrections import get_interpolator
from islatu.io import i07_dat_to_dict_dataframe

if is_DCD_data:
    itp = get_interpolator(DCD_normalization_file, i07_dat_to_dict_dataframe)
    profile.qdcd_normalisation(itp)
    title = "DCD + " + title


In [ ]:
# Now plot the reflectivity curve after correcting for the DCD q-dependent 
# incident intensity.
if is_DCD_data:
    plot_xrr_curve(profile, title=title)

In [ ]:
# Let's make this look a bit more... connected. The reason the XRR curve is
# so bumpy at the moment is because the beam attenuation is changing with scan
# number. Correcting for attenuation variation between scans will help a lot!

# The beam attenuation is stored in the .nxs file that we used to load our 
# profile, right at the beginning. So, islatu already knows about the 
# attenuation as a function of scan number! All we need to do is tell islatu
# to correct for it, and we're done.

profile.transmission_normalisation()

In [ ]:
# Keeping this separate so one can replot the graph without changing the title!
title = "Transmission + " + title

In [ ]:
# Now how transmission normalization has affected the curve.
plot_xrr_curve(profile, title=title)

In [ ]:
# Next, we need to correct for the fact that, at low Q, only a small fraction
# of the beam is incident on the sample surface. At high Q, all of the beam 
# will be incident on the sample surface. This correction is known as a 
# footprint correction, and in Islatu this is handled exactly assuming that the
# beam has a Gaussian profile.

# We will need to specify the fwhm of the beam, in m.
beam_FWHM = 100e-6

# In I07 the trough is around 20cm long.
sample_size = 200e-3

# Carry out the footprint correction.
profile.footprint_correction(beam_width=beam_FWHM, sample_size=sample_size)


In [ ]:
# Change the title accordingly.
title = "Footprint + " + title

In [ ]:
# ... and plot!
plot_xrr_curve(profile, title=title)

In [ ]:
# Now all necessary corrections have been performed, we need to stitch the 
# individual scans together to arrive at one single profile. Islatu can do this
# automatically with its rebin method.
 
# Rebin our data into 500 equally spaced bins. If you want logarithmically 
# spaced bins, call like this:   
# profile.rebin(..., rebin_as="log", ....)

number_of_new_qs = 4000
profile.rebin(number_of_q_vectors=number_of_new_qs)

In [ ]:
# title = "R(Q), Fully Corrected"
# plot_xrr_curve(profile, title=title)

tom_processing_path = data_dir + "processing/"
tom_875 = tom_processing_path + "404875_refl3b.dat"

# Prepare the figure.
fig = go.Figure()

arr = np.loadtxt(tom_875)
x = arr.T[0]
y = arr.T[1]
yerr = arr.T[2]

fig.add_trace(go.Scatter(x=x, y=y, 
    error_y = {"type":'data', "array":yerr, "visible":True}, name="I07"))

fig.add_trace(go.Scatter(x=(profile.q), 
    y=(profile.R), error_y={
        "type":'data', 
        "array":(profile.R_e),
        "visible":True},
    name="Islatu"))



fig.update_yaxes(type="log")

fig.update_layout(title='Islatu vs I07',
    autosize=False,
    width=1000, height=800,
    margin=dict(l=65, r=50, b=65, t=90),
    xaxis_title = "q/Å", yaxis_title = "R")
fig.show()


In [ ]:
print(profile.R_e)

In [ ]:
# FROM HERE ON CODE IS DEPRECATED; KEPT ONLY FOR INFORMATIONAL PURPOSES.
# FROM HERE ON CODE IS DEPRECATED; KEPT ONLY FOR INFORMATIONAL PURPOSES.
# FROM HERE ON CODE IS DEPRECATED; KEPT ONLY FOR INFORMATIONAL PURPOSES.
# FROM HERE ON CODE IS DEPRECATED; KEPT ONLY FOR INFORMATIONAL PURPOSES.
# FROM HERE ON CODE IS DEPRECATED; KEPT ONLY FOR INFORMATIONAL PURPOSES.


# Now the image is cropped, it's time to subtract the background. We'll do this
# by fitting Gaussians to each of the scattering profiles along x. The constant
# offset in the Gaussian fit will be taken to be the background.

from islatu.background import fit_gaussian_1d
importlib.reload(islatu.background)


# It isn't necessary to save the value of the optimized fitting parameters to 
# use Islatu. Here we're keeping track of the popts to check if our Gaussian
# fits look sensible.
popts = profile.bkg_sub(fit_gaussian_1d)


In [ ]:
# Now lets plot the Gaussians and check the fits!
from islatu.background import univariate_normal

# Scan and image numbers of interest.
scan_number = 5
img_number = 5

# Prepare the figure.
title = "Scan {}, Image {}".format(scan_number, img_number)
fig = go.Figure().update_layout(title=title)

# Get the Gaussian fit parameters from the scan of interest.
# popt is: <mean>, <sigma>, <offset>, <factor>.
popt = popts[scan_number][img_number][0]

# Get the image array for the scan of interest.
arr = get_image_array(scan_number, img_number)

arr_along_x = np.mean(arr, axis=0)

# Create x-axes
pixels = np.arange(x_end-x_start)
pseudo_pixels = np.arange(0,  x_end - x_start, 0.05)

# Generate the Gaussian with optimal fitting parameters.
gauss_opt = univariate_normal(pseudo_pixels, *popt)

arr_along_x = np.sqrt(arr_along_x)
gauss_opt = np.sqrt(gauss_opt)


fig.add_trace(go.Scatter(x=pixels, y=arr_along_x))
fig.add_trace(go.Scatter(x=pseudo_pixels, y=gauss_opt, line=dict(dash='dot')))
fig.show()


In [ ]:
# Above we've plotted a single Gaussian. Now lets plot all the fits in one.


In [ ]:
#TODO: this cell is for use in the auto cropping module.

# This gives us the rough location of the peak, but really we want a little more
# detailed information. Lets use scipy's find_peaks function to extract some 
# additional info.
from scipy.signal import find_peaks

# Specifying the prominence of the peak allows us to filter out signal peaks 
# from noise.
prominence = (0.1, None)
x_peaks, properties = find_peaks(raw_frame_x_mean, prominence=prominence)
y_peaks, properties = find_peaks(raw_frame_y_mean, prominence=prominence)

# Prepare the figures as before
fig, axs = plt.subplots(4, 1)

axs[0].plot(raw_frame_x_mean)
axs[1].plot(raw_frame_y_mean)
axs[2].plot(raw_frame_x_mean)
axs[3].plot(raw_frame_y_mean)

# The following is deprecated, for use with matplotlib
figure(figsize=(20, 10), dpi=80)

# Label the peaks
axs[0].plot(x_peaks, raw_frame_x_mean[x_peaks], "x")
axs[1].plot(y_peaks, raw_frame_y_mean[y_peaks], "x")

for i, x_val in enumerate(x_peaks):
    axs[0].annotate(x_val, (x_val, raw_frame_x_mean[x_val]))

for i, x_val in enumerate(y_peaks):
    axs[1].annotate(x_val, (x_val, raw_frame_y_mean[x_val]))



In [ ]:
# Now we want to crop the image such that the peaks shown above are a